In [1]:
from motion.BVH import BVH as BVH
from net.quat import quat
from net.euler import euler
import numpy as np
filename = "./data/Horse/02/run.bvh"
filename2 = "./data/Human/016/16_16.bvh"

#horse_skeleton = BVH.load(filename)     
human_skeleton = BVH.load(filename2)

{'Hips': 'Head_End', 'Spine1': 'RightHandIndex1_End', 'LeftHand': 'LThumb_End', 'RightHand': 'RThumb_End'}
{'Hips': 'Head_End', 'Spine1': 'RightHandIndex1_End', 'LeftHand': 'LThumb_End', 'RightHand': 'RThumb_End'}


KeyError: 1

In [ ]:
def CreateCentaur(human_skeleton, horse_skeleton):
    

In [15]:
skeleton.nodes

{'run': <net.node.node at 0x23e5cdf2650>,
 'Spine_01': <net.node.node at 0x23e5cdf27a0>,
 'Spine_02': <net.node.node at 0x23e5cdf27d0>,
 'Spine_03': <net.node.node at 0x23e5cdf2800>,
 'Spine_04': <net.node.node at 0x23e5cdf2830>,
 'Neck_Base': <net.node.node at 0x23e5cdf2890>,
 'Neck_01': <net.node.node at 0x23e5cdf28c0>,
 'Neck_02': <net.node.node at 0x23e5cdf2860>,
 'Head': <net.node.node at 0x23e5cdf28f0>,
 'L_Ear': <net.node.node at 0x23e5cdf2920>,
 'L_Ear_End': <net.node.node at 0x23e5cdf2950>,
 'R_Ear': <net.node.node at 0x23e5cdf2980>,
 'R_Ear_End': <net.node.node at 0x23e5cdf29b0>,
 'Jaw': <net.node.node at 0x23e5cdf29e0>,
 'Jaw_End': <net.node.node at 0x23e5cdf2a10>,
 'R_Scapula': <net.node.node at 0x23e5cdf2a40>,
 'R_Shoulder': <net.node.node at 0x23e5cdf2a70>,
 'R_Front_UpperLeg': <net.node.node at 0x23e5cdf2aa0>,
 'R_Front_LowerLeg': <net.node.node at 0x23e5cdf2ad0>,
 'R_Front_Ankle': <net.node.node at 0x23e5cdf2b00>,
 'R_Front_Hoof': <net.node.node at 0x23e5cdf2b30>,
 'R_F

In [2]:
skeleton.calculate_global_positions(skeleton.root)

In [10]:
skeleton.root.children[0].positions_local

[array([ 1.16001989e-01,  1.25063493e+02, -4.75769388e+01]),
 array([-90.71312507,  89.14749397, -41.57293782]),
 array([-64.86369553, 101.38568353,  58.46451981]),
 array([ -50.1340751 ,    4.30481722, -123.98588211]),
 array([ -27.22844269, -129.67784391,   18.62054265]),
 array([ -72.46183098, -109.44621779,   25.98584927]),
 array([-109.39398625,   75.79902897,   13.85347899]),
 array([72.37956897, 98.32654086, 54.74984253]),
 array([ -4.45386871, 124.44088376, -48.98045939]),
 array([  -5.49261978, -126.35378658,  -43.69219205]),
 array([-74.37982163, -92.52769289, -61.73106255]),
 array([  78.97694393,   -4.52016479, -107.91972173]),
 array([ 41.66846396, 126.84990089,  -8.79190845]),
 array([ 20.22487175,  67.82871151, 113.55472847]),
 array([111.88260784,  35.31711082,  64.33847718]),
 array([ 97.63987184,  86.08633309, -30.98474757])]

In [11]:
skeleton.root.children[0].positions_global

[array([ 1.18158003e-01,  1.25064776e+02, -4.75735623e+01]),
 array([ -55.87934483,   36.13924639, -116.08578548]),
 array([-128.23551789,   -6.22917388,   37.70020219]),
 array([  -3.82720669, -122.11492954,  -54.5687857 ]),
 array([ 39.92149317, 126.27782078,  19.09560846]),
 array([86.31052043, 90.56143126, 47.47185621]),
 array([-46.73612774,  26.34803616, 122.58046849]),
 array([ 81.76666082, -19.50542476, 104.106712  ]),
 array([ -2.15331023, 122.88122994, -52.91523868]),
 array([ 87.43525285, 100.52826207,  12.39358643]),
 array([-39.05239375,  44.82708075, 119.8745147 ]),
 array([  44.9722378 , -125.9523182 ,   -4.23878132]),
 array([ 64.95948318, 109.84723351,  40.22818335]),
 array([ 47.99598355, -98.16497306,  77.2300429 ]),
 array([  79.73272402,  -22.37416986, -105.10254584]),
 array([  75.551329  ,   17.37163264, -109.0627295 ])]